## Read graph of metafeatures

In [81]:
import networkx as nx
g_x = nx.read_gpickle("encoded_fasttext.gpickle")
#g = nx.read_gpickle("encoded_features.gpickle")
#g = nx.read_gpickle("siimple.gpickle")
order = 0
for x,n in sorted(g_x.nodes(data=True)):
    t = n['tipo']
    if t == "dataset":
        n['tipo'] = 0
    if t == "feature dataset":
        n['tipo'] = 1
    if t == "literal dataset":
        n['tipo'] = 2
    if t == "attribute":
        n['tipo'] = 3
    if t == "feature attribute":
        n['tipo'] = 4
    if t == "literal attribute":
        n['tipo'] = 5  
    n['order']=order
    order+=1

In [82]:
datasets = [x for x,y in g_x.nodes(data=True) if y['tipo']==0]
order = [y['order'] for x,y in g_x.nodes(data=True) if y['tipo']==0]

In [83]:
map_order = dict(zip(datasets,order))

In [84]:
map_order['DS_1']

374

## Get 60% train, 20% val and 20% test from csv

In [85]:
import numpy as np
import pandas as pd

def read_dataset(path,drop_columns=None,keep_columns=None):
    #get rid of useless columns
    csv_data = pd.read_csv(path)
    
    if keep_columns != None:
        #keep only these columns
        return csv_data.filter(items=keep_columns)
    
    if drop_columns!= None:
        #drop these and keep the rest
        return csv_data.drop(drop_columns, axis=1)
    
    #finally, didn't drop or filter any column
    return csv_data     


In [86]:
from random import randrange
def get_samples(df):
    train_mask = []
    val_mask = []
    test_mask = []
    for index, row in df.iterrows():
        if row[2] == 1:
            if randrange(0, 10) <= 7:
                train_mask.append([row[0],row[1],row[2]])
            else:
                test_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
#                 if index % 2 == 0:
#                     val_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
#                 else:
#                     test_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
        else:
            if randrange(0, 10) > 8:
#                 if randrange(0, 30) <= 21:
                if randrange(0, 30) <= 24:
                    train_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
                else:
                    test_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
#                     if index % 2 == 0:
#                         val_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
#                     else:
#                         test_mask.append([map_order['DS_'+str(row[0])],map_order['DS_'+str(row[1])],row[2]])
    return train_mask,test_mask
#     return train_mask,val_mask,test_mask

In [87]:
df_matching = read_dataset("./openml_203ds_datasets_matching.csv",keep_columns=["'dataset1_id'", "'dataset2_id'","'matching_topic'"]);
# train_mask,val_mask,test_mask = get_samples(df_matching)
# print("Train samples: "+str(len(train_mask)) + " Val samples: "+str(len(val_mask))+ " Test samples: "+str(len(test_mask)))
train_mask,test_mask = get_samples(df_matching)
print("Train samples: "+str(len(train_mask)) + " Test samples: "+str(len(test_mask)))


Train samples: 2050 Test samples: 452


In [88]:
test_mask

[[374, 501, 0],
 [374, 386, 0],
 [374, 544, 0],
 [571, 566, 0],
 [571, 405, 0],
 [571, 432, 0],
 [571, 444, 0],
 [571, 450, 0],
 [571, 533, 0],
 [571, 514, 0],
 [576, 541, 1],
 [576, 391, 0],
 [576, 435, 1],
 [576, 512, 1],
 [576, 484, 1],
 [576, 528, 0],
 [417, 464, 1],
 [417, 496, 1],
 [417, 556, 1],
 [417, 382, 1],
 [417, 384, 1],
 [417, 531, 0],
 [418, 464, 1],
 [418, 496, 1],
 [418, 508, 1],
 [418, 393, 0],
 [418, 441, 1],
 [464, 536, 0],
 [464, 567, 0],
 [464, 382, 1],
 [464, 379, 0],
 [466, 471, 0],
 [466, 551, 0],
 [466, 395, 0],
 [466, 416, 0],
 [470, 558, 0],
 [470, 382, 1],
 [470, 384, 1],
 [470, 379, 0],
 [476, 496, 0],
 [476, 469, 0],
 [476, 543, 0],
 [476, 382, 0],
 [476, 407, 0],
 [476, 414, 0],
 [476, 456, 0],
 [476, 436, 0],
 [486, 501, 0],
 [486, 447, 0],
 [486, 547, 0],
 [486, 383, 0],
 [486, 440, 0],
 [486, 441, 1],
 [491, 535, 0],
 [491, 441, 0],
 [495, 521, 0],
 [495, 517, 0],
 [495, 541, 1],
 [495, 574, 1],
 [496, 556, 1],
 [496, 385, 0],
 [496, 396, 0],
 [496, 4

## Deep graph library

In [89]:
import dgl
#convert from networkx to graph deep library format
g = dgl.DGLGraph()
#gdl.from_networkx(g,['vector'])
g.from_networkx(g_x,node_attrs=['tipo','vector'], edge_attrs=None)

Using backend: pytorch


In [90]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

gcn_msg = fn.copy_src(src='vector', out='m')
gcn_reduce = fn.sum(msg='m', out='vector')

In [91]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g,feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['vector'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['vector']
            return self.linear(h)

In [92]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(300, 300)
        self.layer2 = GCNLayer(300, 300)
    
    def forward(self, g,features):
        x = F.relu(self.layer1(g, features))
        x = F.relu(self.layer2(g, x))
        return x
net = Net()
print(net)

Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=300, out_features=300, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=300, out_features=300, bias=True)
  )
)


In [93]:
from scipy.spatial.distance import cosine

def cosine_vectors(vec1,vec2):
    return (1- cosine(vec1,vec2))

def resultSet_train(features,mask):
    indices = []
    labels = []
    for n in mask:
        resultCos = cosine(features[mask[0]],features[mask[1]])
        # index 0 is not similar, index 1 is similar
        result = th.tensor([1 - resultCos , resultCos])
        indices.append(result)
        labels.append(mask[2])
    return indices,labels


def resultSet_eval(features,mask):
    indices = []
    labels = []
    for n in mask:
        resultCos = cosine(features[mask[0]],features[mask[1]])
        # index 0 is not similar, index 1 is similar
        result = th.tensor([1 - resultCos , resultCos])
        # get the index that is greater
        out = th.max(result, dim=1)
        indices.append(out)
        labels.append(mask[2])
    return indices,labels

def evaluate(model, g, features, mask):
    model.eval()
    with th.no_grad():
        embeddings = model(g, features)
        indices , labels = resultSet_eval(embeddings,mask)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [ ]:
import time
import numpy as np
net = Net()
#g, features, labels, train_mask, test_mask

optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
for epoch in range(5):
    if epoch >=3:
        t0 = time.time()

    net.train()
    embeddings = net(g, g.ndata['vector'])
    indices,labels = resultSet_train(embeddings,train_mask)
    loss = F.nll_loss(indices, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if epoch >=3:
        dur.append(time.time() - t0)
    
    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

## Tests

In [ ]:
#get data from a specific node
g.nodes[0].data
#get data from nodes
g.ndata
#another way of accessing data from a node
g.ndata['tipo'][0]

In [ ]:
g.edges()

In [ ]:
g.has_edge_between(374,17619)

In [ ]:
g.ndata

In [ ]:
for n in g.nodes:
    print (n.data['tipo'])